## Introduction
This notebook demonstrates the full interface of the `forecast()` function. 

The best known and most frequent usage of `forecast` enables forecasting on test sets that immediately follows training data. 

However, in many use cases it is necessary to continue using the model for some time before retraining it. This happens especially in **high frequency forecasting** when forecasts need to be made more frequently than the model can be retrained. Examples are in Internet of Things and predictive cloud resource scaling.

Here we show how to use the `forecast()` function when a time gap exists between training data and prediction period.

Terminology:
* forecast origin: the last period when the target value is known
* forecast periods(s): the period(s) for which the value of the target is desired.
* lookback: how many past periods (before forecast origin) the model function depends on. The larger of number of lags and length of rolling window.
* prediction context: `lookback` periods immediately preceding the forecast origin

## Setup

1. **Model**  
   We will need the MLflow model, which is downloaded at the end of the training notebook. Follow any training notebook to get the model. The MLflow model is usually downloaded to the folder: `./artifact_downloads/outputs/mlflow-model`.

2. **Environment**  
   We will need the environment to load the model. Please run the following commands to create the environment (the conda file is usually downloaded to: `./artifact_downloads/outputs/mlflow-model/conda.yaml`):
   - `conda env create --file <path_to_conda_yaml>`
   - `conda activate project_environment`

3. **Register environment as kernel**  
   - Please run the following command to register the environment as a kernel:  
     ```bash
     python -m ipykernel install --user --name project_environment --display-name "model-inference"
     ```
   - Refresh the kernel and then select the newly created kernel named `model-inference` from the kernel dropdown.
   
   Now we are good to run this notebook in the newly created kernel.


In [32]:
TIME_COLUMN_NAME = "date"
TIME_SERIES_ID_COLUMN_NAME = "time_series_id"
TARGET_COLUMN_NAME = "y"
lags = [1, 2, 3]
forecast_horizon = 6

In [33]:
import mlflow.pyfunc
import mlflow.sklearn
import pandas as pd

df_train = pd.read_parquet('./data/training-mltable-folder/df_train.parquet') # We stored the training and test data during training
df_test = pd.read_parquet('./data/testing-mltable-folder/df_test.parquet')

### Batch Inferencing

In [34]:
import mlflow
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
credential = DefaultAzureCredential()
ml_client = None

subscription_id = "72c03bf3-4e69-41af-9532-dfcdc3eefef4"#"<SUBSCRIPTION_ID>"
resource_group = "aml-benchmarking"#"<RESOURCE_GROUP>"
workspace = "aml-benchmarking-rd"#"<AML_WORKSPACE_NAME>"

ml_client = MLClient(credential, subscription_id, resource_group, workspace)

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourceGroups/aml-benchmarking/providers/Microsoft.MachineLearningServices/workspaces/aml-benchmarking-rd


In [35]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))

from mlflow.tracking.client import MlflowClient
from mlflow.artifacts import download_artifacts

# Initialize MLFlow client
mlflow_client = MlflowClient()


Current tracking uri: azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourceGroups/aml-benchmarking/providers/Microsoft.MachineLearningServices/workspaces/aml-benchmarking-rd


In [36]:
# job_name = returned_job.name
# Example if providing an specific Job name/ID
job_name = "yellow_camera_1n84g0vcwp"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

Parent Run: 
<Run: data=<RunData: metrics={'explained_variance': 0.9910013504019911,
 'mean_absolute_error': 0.29282333566680235,
 'mean_absolute_percentage_error': 1.118067051888466,
 'median_absolute_error': 0.3040329750308685,
 'normalized_mean_absolute_error': 0.011402639389141949,
 'normalized_median_absolute_error': 0.011167830489974646,
 'normalized_root_mean_squared_error': 0.013701302313198613,
 'normalized_root_mean_squared_log_error': 0.00621450456108454,
 'r2_score': 0.9863519731383007,
 'root_mean_squared_error': 0.35499956438249874,
 'root_mean_squared_log_error': 0.01294906396693142,
 'spearman_correlation': 1.0}, params={}, tags={'automl_best_child_run_id': 'yellow_camera_1n84g0vcwp_4',
 'fit_time_000': '0.891015;0.8447906666666666;0.880821;0.8549053333333333;0.9084923333333333;1.1095196666666667;1.1935733333333334;0.029423;0.4111286666666667;0.007625666666666667;0.009171;0.008983;0.008941333333333334;0.05766866666666667;0.008226666666666667;0.008711666666666666;0.00838

In [37]:
# Get the best model's child run
best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

Found best child run id:  yellow_camera_1n84g0vcwp_4
Best child run: 
<Run: data=<RunData: metrics={'explained_variance': 0.9910013504019911,
 'mean_absolute_error': 0.29282333566680235,
 'mean_absolute_percentage_error': 1.118067051888466,
 'median_absolute_error': 0.3040329750308685,
 'normalized_mean_absolute_error': 0.011402639389141949,
 'normalized_median_absolute_error': 0.011167830489974646,
 'normalized_root_mean_squared_error': 0.013701302313198613,
 'normalized_root_mean_squared_log_error': 0.00621450456108454,
 'r2_score': 0.9863519731383007,
 'root_mean_squared_error': 0.35499956438249874,
 'root_mean_squared_log_error': 0.01294906396693142,
 'spearman_correlation': 1.0}, params={}, tags={'mlflow.parentRunId': 'yellow_camera_1n84g0vcwp',
 'mlflow.rootRunId': 'yellow_camera_1n84g0vcwp',
 'mlflow.runName': 'ivory_bear_5j8y7myk',
 'mlflow.user': 'Sampurna Goswami',
 'model_explain_run_id': 'yellow_camera_1n84g0vcwp_ModelExplain',
 'model_explanation': 'True'}>, info=<RunInfo:

In [38]:
import datetime
from azure.ai.ml.entities import (
    Environment,
    BatchEndpoint,
    BatchDeployment,
    BatchRetrySettings,
    Model,
)
from azure.ai.ml.constants import BatchDeploymentOutputAction

model_name = "test-gap-batch-endpoint"
batch_endpoint_name = "gap-batch-" + datetime.datetime.now().strftime(
    "%m%d%H%M%f"
)

model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/model.pkl",
    name=model_name,
    description="Gap prediction sample best model",
)
registered_model = ml_client.models.create_or_update(model)

env = Environment(
    name="automl-tabular-env",
    description="environment for automl inference",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    conda_file="artifact_downloads/outputs/conda_env_v_1_0_0.yml",
)

endpoint = BatchEndpoint(
    name=batch_endpoint_name,
    description="this is a sample batch endpoint",
)
ml_client.begin_create_or_update(endpoint).wait()

In [39]:
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.ml.entities import AmlCompute

cluster_name = "gap-cluster"

try:
    # Retrieve an already attached Azure Machine Learning Compute.
    compute = ml_client.compute.get(cluster_name)
except ResourceNotFoundError as e:
    compute = AmlCompute(
        name=cluster_name,
        size="STANDARD_DS12_V2",
        type="amlcompute",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=120,
    )
    poller = ml_client.begin_create_or_update(compute)
    poller.wait()


In [40]:
batch_endpoint_name

'gap-batch-10241739762384'

In [42]:
output_file = "forecast.csv"
batch_deployment = BatchDeployment(
    name="oj-non-mlflow-deployment",
    description="this is a sample non-mlflow deployment",
    endpoint_name=batch_endpoint_name,
    model=registered_model,
    code_path="./forecasting_script",
    scoring_script="forecasting_script.py",
    environment=env,
    environment_variables={
        "TARGET_COLUMN_NAME": TARGET_COLUMN_NAME,
    },
    compute=cluster_name,
    instance_count=1, #2
    max_concurrency_per_instance=1, #2
    mini_batch_size=1, #10
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name=output_file,
    retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
    logging_level="info",
    properties={"include_output_header": "true"},
    tags={"include_output_header": "true"},
)


In [43]:
ml_client.begin_create_or_update(batch_deployment).wait()

In [44]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
my_test_data_input = Input(
    type=AssetTypes.URI_FOLDER,
    path="./data/testing-mltable-folder",
)

In [45]:
ml_client

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x0000029EE13BDB20>,
         subscription_id=72c03bf3-4e69-41af-9532-dfcdc3eefef4,
         resource_group_name=aml-benchmarking,
         workspace_name=aml-benchmarking-rd)

In [46]:
batch_endpoint_name

'gap-batch-10241739762384'

In [47]:
my_test_data_input

{'type': 'uri_folder', 'path': './data/testing-mltable-folder'}

In [48]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=batch_endpoint_name,
    input=my_test_data_input,
    deployment_name="oj-non-mlflow-deployment",  # name is required as default deployment is not set
)

In [49]:
job_name = job.name
batch_job = ml_client.jobs.get(name=job_name)
print(batch_job.status)
# stream the job logs
ml_client.jobs.stream(name=job_name)

Running
RunId: batchjob-f47ea9fe-132f-4b91-b038-ca19fb71c9be
Web View: https://ml.azure.com/runs/batchjob-f47ea9fe-132f-4b91-b038-ca19fb71c9be?wsid=/subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourcegroups/aml-benchmarking/workspaces/aml-benchmarking-rd

Streaming logs/azureml/executionlogs.txt

[2024-10-24 12:12:27Z] Submitting 1 runs, first five are: 16057eb9:658e18fb-67ae-4a36-bbdf-fd8db589f027
[2024-10-24 12:20:40Z] Execution of experiment failed, update experiment status and cancel running nodes.

Execution Summary
RunId: batchjob-f47ea9fe-132f-4b91-b038-ca19fb71c9be
Web View: https://ml.azure.com/runs/batchjob-f47ea9fe-132f-4b91-b038-ca19fb71c9be?wsid=/subscriptions/72c03bf3-4e69-41af-9532-dfcdc3eefef4/resourcegroups/aml-benchmarking/workspaces/aml-benchmarking-rd


JobException: Exception : 
 {
    "error": {
        "code": "UserError",
        "message": "Pipeline has failed child jobs. For more details and logs, please go to the job detail page and check the child jobs.",
        "message_format": "Pipeline has failed child jobs. {0}",
        "message_parameters": {},
        "reference_code": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "eastus",
    "location": "eastus",
    "time": "2024-10-24T12:20:40.0429Z",
    "component_name": ""
} 

# Local inferencing from model pickle


In [3]:
import mlflow.pyfunc
import mlflow.sklearn
# Please ensure that the training artifacts are downloaded. For more details refer to the training notebook
mlflow_dir = "./artifact_downloads/outputs/mlflow-model"
fitted_model = mlflow.sklearn.load_model(mlflow_dir)

ModuleNotFoundError: No module named 'azureml.training'

In [ ]:
df_train[df_train['time_series_id']=="ts1"].tail(2)

In [ ]:
df_test[df_test['time_series_id']=="ts1"].head(2)

# Forecasting from the trained model

In this section we will review the forecast interface for two main scenarios: forecasting right after the training data, and the more complex interface for forecasting when there is a gap (in the time sense) between training and testing data.

## X_train is directly followed by the X_test
Let's first consider the case when the prediction period immediately follows the training data. This is typical in scenarios where we have the time to retrain the model every time we wish to forecast. Forecasts that are made on daily and slower cadence typically fall into this category. Retraining the model every time benefits the accuracy because the most recent data is often the most informative.


<img src="./images/forecast_function_at_train.png" alt="Description" width="50%">

We use X_test as a forecast request to generate the predictions.

In [ ]:
X_test = df_test.copy()
y_test = X_test.pop(TARGET_COLUMN_NAME).values.astype(float)

y_pred_no_gap, xy_nogap = fitted_model.forecast(X_test)

### Confidence Intervals
Forecasting model may be used for the prediction of forecasting intervals by running forecast_quantiles(). This method accepts the same parameters as forecast().

In [ ]:
quantiles = fitted_model.forecast_quantiles(X_test)
quantiles

### Distribution forecasts
Often the figure of interest is not just the point prediction, but the prediction at some quantile of the distribution. This arises when the forecast is used to control some kind of inventory, for example of grocery items or virtual machines for a cloud service. In such case, the control point is usually something like "we want the item to be in stock and not run out 99% of the time". This is called a "service level". Here is how you get quantile forecasts.

In [ ]:
# Specify which quantiles you would like
fitted_model.quantiles = [0.01, 0.5, 0.95]

# use forecast_quantiles function, not the forecast() one
y_pred_quantiles = fitted_model.forecast_quantiles(X_test)

# quantile forecasts returned in a Dataframe along with the time and time series id columns
y_pred_quantiles

# Forecasting away from training data
Suppose we trained a model, some time passed, and now we want to apply the model without re-training. If the model "looks back" -- uses previous values of the target -- then we somehow need to provide those values to the model.

<img src="./images/forecast_function_away_from_train.png" alt="Description" width="50%">

The notion of forecast origin comes into play: **the forecast origin is the last period for which we have seen the target value.** This applies per time-series, so each time-series can have a different forecast origin.

The part of data before the forecast origin is the **prediction context**. To provide the context values the model needs when it looks back, we pass definite values in y_test (aligned with corresponding times in X_test).

In [ ]:
# generate the same kind of test data we trained on, but now make the train set much longer, so that the test set will be in the future
from helper import get_timeseries, make_forecasting_query
X_context, y_context, X_away, y_away = get_timeseries(
    train_len=42,  # train data was 30 steps long
    test_len=4,
    time_column_name=TIME_COLUMN_NAME,
    target_column_name=TARGET_COLUMN_NAME,
    time_series_id_column_name=TIME_SERIES_ID_COLUMN_NAME,
    time_series_number=2,
)

print("End of the data we trained on:")
print(df_train.groupby(TIME_SERIES_ID_COLUMN_NAME)[TIME_COLUMN_NAME].max())

print("\nStart of the data we want to predict on:")
print(X_away.groupby(TIME_SERIES_ID_COLUMN_NAME)[TIME_COLUMN_NAME].min())

There is a gap of 12 hours between end of training and beginning of X_away. (It looks like 13 because all timestamps point to the start of the one hour periods.) Using only X_away will fail without adding context data for the model to consume

In [ ]:
try:
    y_pred_away, xy_away = fitted_model.forecast(X_away)
    xy_away
except Exception as e:
    print(e)

How should we read that eror message? The forecast origin is at the last time the model saw an actual value of y (the target). That was at the end of the training data! The model is attempting to forecast from the end of training data. But the requested forecast periods are past the forecast horizon. We need to provide a define y value to establish the forecast origin.

We will use the helper function to take the required amount of context from the data preceding the testing data. It's definition is intentionally simplified to keep the idea in the clear.

Let's see where the context data ends - it ends, by construction, just before the testing data starts.

In [ ]:
print(
    X_context.groupby(TIME_SERIES_ID_COLUMN_NAME)[TIME_COLUMN_NAME].agg(
        ["min", "max", "count"]
    )
)
print(
    X_away.groupby(TIME_SERIES_ID_COLUMN_NAME)[TIME_COLUMN_NAME].agg(
        ["min", "max", "count"]
    )
)
X_context.tail(5)

How should we read that eror message? The forecast origin is at the last time the model saw an actual value of y (the target). That was at the end of the training data! The model is attempting to forecast from the end of training data. But the requested forecast periods are past the forecast horizon. We need to provide a define y value to establish the forecast origin.

We will use this helper function to take the required amount of context from the data preceding the testing data. It's definition is intentionally simplified to keep the idea in the clear.

In [ ]:
# Since the length of the lookback is 3, we need to add 3 periods from the context to the request so that the model has the data it needs

# Put the X and y back together for a while. They like each other and it makes them happy.
X_context[TARGET_COLUMN_NAME] = y_context
X_away[TARGET_COLUMN_NAME] = y_away
fulldata = pd.concat([X_context, X_away])

# Forecast origin is the last point of data, which is one 1-hr period before test
forecast_origin = X_away[TIME_COLUMN_NAME].min() - pd.DateOffset(hours=1)
# it is indeed the last point of the context
assert forecast_origin == X_context[TIME_COLUMN_NAME].max()
print("Forecast origin: " + str(forecast_origin))

# The model uses lags and rolling windows to look back in time
n_lookback_periods = max(lags) # n_lookback_periods = max(max(lags), forecast_horizon) # If target_rolling_window_size is used
lookback = pd.DateOffset(hours=n_lookback_periods)
horizon = pd.DateOffset(hours=forecast_horizon)

In [ ]:
# now make the forecast query from context (refer to figure)
X_pred, y_pred = make_forecasting_query(
    fulldata, TIME_COLUMN_NAME, TARGET_COLUMN_NAME, forecast_origin, horizon, lookback
)

# show the forecast request aligned
X_show = X_pred.copy()
X_show[TARGET_COLUMN_NAME] = y_pred
X_show[X_show['time_series_id']=="ts0"]

In [ ]:
X_pred['data_type']="unknown"   # Our trining had an additional column called data_type, hence, adding it

In [ ]:
# Now everything should work
y_pred_away, xy_away = fitted_model.forecast(X_pred, y_pred)

# show the forecast aligned without the generated features
X_show = xy_away.reset_index()
X_show[["date", "time_series_id", "ext_predictor", "_automl_target_col"]] # prediction is in _automl_target_col

In [ ]:
### Let us look at the tail of training data and the head of the test data for one grain

In [ ]:
df_train[df_train['time_series_id']=="ts1"].tail(2)

If there is a gap between the train and the test data, and the test data uses lags/ rolling forecasts, we need to append the context data such that the test data has access to the lags
In the above case, train_data ends at 2000-01-02 05:00:00

In [ ]:
X_show[X_show['time_series_id'] == "ts1"][["date", "time_series_id", "ext_predictor", "_automl_target_col"]]

### Forecasting using batch endpoint